In [9]:
import os.path
import subprocess
import configparser
from pytube import YouTube
from pytube import Playlist
from tkinter import *
from tkinter import ttk, messagebox
from tkinter.filedialog import askopenfilename, askdirectory

def create():
    conf = configparser.ConfigParser()
    conf['DEFAULTS'] = {'SaveDirectory':''}
    with open('configuration.ini','w') as file:
        conf.write(file)
        
def read():
    if not os.path.isfile('configuration.ini'):
        createConfigfile()
    conf = configparser.ConfigParser()
    conf.read('configuration.ini')
    local = conf['DEFAULTS']['SaveDirectory']
    return local

def save():
    localdir = myDirectoryName.get()
    conf = configparser.ConfigParser()
    conf['DEFAULTS'] = {'SaveDirectory':localdir}
    with open('configuration.ini','w') as file:
        conf.write(file)

def select():
    ask = askdirectory()
    myDirectoryName.set(ask)

def downloadVideo():
    video = youtubelink.get()
    content = YouTube(video)
    stream = content.streams.first()
    stream.download(myDirectoryName.get())
    saveConfigfile()
    messagebox.showinfo("Video Downloaded")
    
def downloadAudio():
    video = youtubelink.get()
    content = YouTube(video)
    stream = content.streams.first()
    stream.download(myDirectoryName.get())
    mp4 = '"'+myDirectoryName.get()+'/'+stream.default_filename+'"'
    mp3 = '"'+myDirectoryName.get()+'/'+content.title+'.mp3'+'"'
    ffmpeg = ('ffmpeg -i %s ' % mp4 + mp3) 
    subprocess.run(ffmpeg)
    os.remove(myDirectoryName.get()+'/'+stream.default_filename)
    saveConfigfile()
    messagebox.showinfo("Video as Audio Downloaded")

def downloadPlaylist():
    pl = Playlist(youtubeplaylistlink.get())
    pl.download_all(myDirectoryName.get())
    saveConfigfile()
    messagebox.showinfo("All Playlist Downloaded")  

def downloadPlaylistMP3():
    pl = Playlist(youtubeplaylistlink.get())
    pl.download_all(myDirectoryName.get())
    directory = os.fsencode(myDirectoryName.get())
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        mp4 = myDirectoryName.get()+'/'+filename
        mp3 = myDirectoryName.get()+'/'+os.path.splitext(filename)[0]+'.mp3'
        ffmpeg = ('ffmpeg -i "%s" "%s"' % (mp4 , mp3)) 
        subprocess.run(ffmpeg)
        os.remove(mp4)
        #messagebox.showinfo("Dateiname",ffmpeg)
    saveConfigfile()
    messagebox.showinfo("Playlist as Audio Downloaded")  


def hint():
    messagebox.showinfo("Hint","After pressing button for download, wait till get notified for completion!")

    
root = Tk()
root.title("Youtube Downloader")

mainframe = ttk.Frame(root, padding="3 3 12 12")
mainframe.grid(column=0, row=0, sticky=(N, W, E, S))
root.columnconfigure(0, weight=1)
root.rowconfigure(0, weight=1)

youtubelink = StringVar()
youtubeplaylistlink = StringVar()
myDirectoryName = StringVar()

tempdirectory = read()
myDirectoryName.set(tempdirectory)

ttk.Label(mainframe,text="Enter the link for YouTube Playlist: ").grid(column=1, row=0,sticky=(W))

YoutubePL_entry = ttk.Entry(mainframe, width = 40, textvariable = youtubeplaylistlink)
YoutubePL_entry.grid(column=2, row=0,sticky=(W,E))

ttk.Button(mainframe, text="Download Video (mp4) ", command=downloadPlaylist).grid(column=3,row=0, sticky=(E,W),pady=10,padx=10)

ttk.Button(mainframe, text="Download Audio (mp3) ", command=downloadPlaylistMP3).grid(column=4,row=0, sticky=(E,W),pady=10,padx=10)

ttk.Label(mainframe,text="Enter YouTube Video Link: ").grid(column=1, row=1,sticky=(W))

Youtube_entry = ttk.Entry(mainframe, width = 40, textvariable = youtubelink)
Youtube_entry.grid(column=2, row=1,sticky=(W,E))

ttk.Button(mainframe, text="Download Video (mp4) ", command=downloadVideo).grid(column=3,row=1, sticky=(E,W),pady=10,padx=10)

ttk.Button(mainframe, text="Download Audio (mp3) ", command=downloadAudio).grid(column=4,row=1, sticky=(E,W),pady=10,padx=10)

ttk.Label(mainframe,text="Set Location for Download").grid(column=1, row=2,sticky=(W))
directoryname_entry = ttk.Entry(mainframe, width = 40, textvariable = myDirectoryName)
directoryname_entry.grid(column=2, row=2,sticky=(W,E))

ttk.Button(mainframe, text="Browse", command=select).grid(column=3, row=2, sticky=W,pady=10,padx=10)
ttk.Button(mainframe, text="Hint", command=hint).grid(column=1, row=3, sticky=W,pady=10,padx=10)

root.mainloop()